In [10]:
import math
import torch
import torch.nn as nn

import torchsummary as ts
import torchinfo as ti

# BERT 구현
### 학습 목표
1. BERT의 Embeddings모듈 동작을 이해하고 구현할 수 있다.
2. BERT의 Self-Attention을 활용한 Transformer 부분인 BertLayer모듈의 동작을 이해하고 구현할 수 있다.
3. BERT의 Pooler모듈의 동작을 이해하고 구현할 수 있다.

## 8.2.2 BERT_Base의 네트워크 설정 파일 읽기
- 먼저 BERT_Base에서 Transformer가 12단인 것과 특징량 벡터가 768차원인 것 등을 적은 weights폴더의 네트워크 설정 파일 bert_config.json을 읽어들입니다.
- 읽어들인 JSON파일의 사전형 변수에서 key 'hidden'값을 취하려면 config['hidden_size']로 적어야합니다. 이를 config.hidden_size로 기술하면 깔끔합니다.

In [4]:
# config.json에서 설정을 읽어들여 JSON 사전 변수를 오브젝트 변수로 변환
import json

config_file = './weights/bert_config.json'

# 파일을 열어 JSON으로 읽는다.
with open(config_file, 'r') as f:
    config = json.load(f)
config

{'architectures': ['BertForMaskedLM'],
 'attention_probs_dropout_prob': 0.1,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'transformers_version': '4.6.0.dev0',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [5]:
# 사전 변수를 오브젝트 변수로
from attrdict import AttrDict

config = AttrDict(config)
config.hidden_size

768

In [7]:
config.vocab_size

30522

## 8.2.3 BERT에 레이어 정규화 층 정의
- BERT 모델 구축의 사전 준비로 레이어 정규화 층의 클래스를 정의합니다. 7장에서 사용한 것 처럼 파이토치에도 레이어 정규화가 있습니다.
- 텐서플로와 파이토치에서는 레이어 정규화의 구현 방법이 약간 다릅니다. 텐서의 마지막 채널(즉 단어의 특징량 벡터 768차원)에 평균 0, 표준편차 1이 되도록 레이어 정규화를 수행합니다. 0으로 나누지 않도록 보조 항 엡실론을 넣는 방법은 파이토치와 텐서플로가 서로 다릅니다.
- 이번에 사용할 학습된 모델은 구글이 공개한 텐서플로의 학습 결과에 기반하여 텐서플로 버전의 레이어 정규화 층을 만듭니다.

In [8]:
# BERT용으로 레이어 정규화 층 정의
# 세부 구현을 텐서플로에 맞춘다.
import torch.nn as nn
import torch

class BertLayerNorm(nn.Module):
    '''레이어 정규화 층'''
    
    def __init__(self, hidden_size, eps=1e-12):
        super(BertLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(hidden_size)) # weight에 대한 것
        self.beta = nn.Parameter(torch.zeros(hidden_size)) # 바이어스에 대한 것
        self.variance_epsilon = eps
    
    def forward(self, x):
        # 평균
        u = x.mean(-1, keepdim=True)
        # 분산
        s = (x - u).pow(2).mean(-1, keepdim=True)
        # 일반 정규화 적용 variance_epsilon은 0으로 나누지 않도록 보조 항 삽입
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        # 최종적으로 gamma 와 x를 곱하고 beta를 더해서 return
        return self.gamma * x + self.beta

In [12]:
model = BertLayerNorm(config.hidden_size, eps=1e-12)
ti.summary(model)

Layer (type:depth-idx)                   Param #
BertLayerNorm                            1,536
Total params: 1,536
Trainable params: 1,536
Non-trainable params: 0

In [26]:
# ts.summary(model, input_size=(10, 768))

## 8.2.4 Embedding 구현
### Transformer의 Embeddings 모듈과 두 가지 큰 차이점 존재
- 첫째, Positional Embedding(위치 정보를 벡터로 변환)의 표현 기법을 Transformer는 sin, cos으로 계산하지만 BERT는 표현 방법도 학습시킵니다. 학습 시키는 것은 단어의 위치 정보뿐이며 단어 벡터의 차원 정보는 부여하지 않습니다. 즉 첫 번째 단어의 768차원은 동일한 position_embeddings값이 저장 되고 두 번째 단어는 첫 번째 단어와는 다르지만 768차원 방향에 같은 position_embeddings값이 저장됩니다.
- 둘째, Sentence Embedding의 존재입니다. BERT는 두 문장을 입력합니다. 첫 번째 문장과 두 번째 문장을 구분하기 위한 Embedding를 준비합니다. Embeddings 모듈에서는 Token Embedding, Positional Embedding, Sentence Embedding에서 각각 구할 세 개의 텐서를 Transformer처럼 더하여 Embeddings 모듈의 출력으로 합니다. Embeddings모듈에 대한 입력 텐서는 (batch_size, seq_len)크기로 이루어진 문장의 단어 ID 나열인 변수 input_ids와 (batch_size, seq_len)의 각 단어가 첫 번째 문장인지 두 번째 문장인지 나타내는 문장 id인 변수 token_type_ids가 됩니다. 출력은 (batch_size, seq_len, hidden_size)의 텐서입니다. seq_len은 512이고 hidden_size는 768입니다.

In [27]:
# BERT의 Embeddings 모듈
class BertEmbeddings(nn.Module):
    '''문장의 단어 ID열과 첫 번째인지 두 번째 문장인지 정보를 내장 벡터로 변환'''
    
    def __init__(self, config):
        super(BertEmbeddings, self).__init__()
        
        # 세 개의 벡터 표현 내장
        
        # Token Embedding: 단어 ID를 단어 벡터로 변환
        # vocab_size = 30522로 BERT의 학습된 모델에 사용된 vocabulary 양
        # hidden_size = 768로 특징량 벡터의 길이는 768
        self.word_embeddings = nn.Embedding(
            config.vocab_size, config.hidden_size, padding_idx=0)
        
        # padding_idx = 0의 idx = 0 단어 벡터는 0으로 한다. BERT의 vocabulary의 idx=0은 [PAD]이다
        
        # Transformer Positional Embedding: 위치 정보 텐서를 벡터로 변환
        # Transformer의 경우는 sin, cos로 이루어진 고정 값이지만 BERT는 학습시킨다.
        # max_position_embeddings = 512로 문장 길이는 512단어
        self.position_embeddings = nn.Embedding(
            config.max_position_embeddings, config.hidden_size)
        
        # Sentence Embedding: 첫 번째, 두 번째 문장을 벡터로 변환
        # type_vocab_size = 2
        self.token_type_embeddings = nn.Embedding(
            config.type_vocab_size, config.hidden_size)
        
        # 작성한 레이어 정규화 층
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        
        # 드롭아웃 'hidden_dropout_prob':0.1
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, input_ids, token_type_ids=None):
        '''
        input_ids: [batch_size, seq_len] 문장의 단어 ID 나열
        token_type_ids: [batch_size, seq_len] 각 단어가 첫 번째 문장인지 두 번째 문장인지 나타내는 id
        '''
        
        # 1. Token Embeddings
        # 단어 ID를 단어 벡터로 변환
        words_embeddings = self.word_embeddings(input_ids)
        
        # 2. Sentence Embeddings
        # token_type_ids가 없는 경우는 문장의 모든 단어를 첫 번째 문장으로 하여 0으로 설정
        # input_ids와 같은 크기로 제로 텐서 작성
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        
        # 3. Transformer Positional Embedding:
        # [0, 1, 2, ...]로 문장의 길이 만큼 숫자가 하나씩 올라간다.
        # [batch_size, seq_len]의 텐서 positional_ids 작성
        # positional_ids를 입력하여 position_embeddings 층에서 768차원의 텐서를 꺼낸다
        seq_length = input_ids.size(1) # 문장 길이
        position_ids = torch.arange(
            seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        
        # 세 개의 내장 텐서를 더한다. [batch_size, seq_len, hidden_size]
        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        
        # 레이어 정규화와 드롭아웃 실행
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        
        return embeddings

In [30]:
model = BertEmbeddings(config)
ti.summary(model)

Layer (type:depth-idx)                   Param #
BertEmbeddings                           --
├─Embedding: 1-1                         23,440,896
├─Embedding: 1-2                         393,216
├─Embedding: 1-3                         1,536
├─BertLayerNorm: 1-4                     1,536
├─Dropout: 1-5                           --
Total params: 23,837,184
Trainable params: 23,837,184
Non-trainable params: 0

In [34]:
input_ids = torch.LongTensor([[31, 51, 12, 23, 99], [15, 5, 1, 0, 0]])
input_ids.shape

torch.Size([2, 5])

In [57]:
token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])
token_type_ids.shape

torch.Size([2, 5])

In [42]:
model(input_ids)

tensor([[[-0.5864,  0.7621,  0.0181,  ..., -1.5074,  1.7349,  0.7103],
         [ 0.3085,  0.9922,  0.5774,  ..., -3.0212,  2.4898, -0.7666],
         [ 0.0000,  0.2328,  0.0461,  ...,  0.0000,  1.9305,  2.1867],
         [ 0.0000, -0.0000,  1.3760,  ..., -0.2753,  0.0000,  1.4326],
         [ 0.6090, -0.5061,  1.1534,  ..., -0.5636,  2.5507,  0.1521]],

        [[ 0.4883,  0.3363,  0.5141,  ..., -1.7378,  0.0000,  2.2326],
         [ 0.2567, -0.1650, -0.1615,  ..., -0.0000,  2.5798, -0.1563],
         [ 0.1157,  1.1629,  0.0148,  ...,  0.7537,  1.2637,  3.1691],
         [-0.5881,  0.3833,  1.7950,  ..., -0.2181,  2.2109,  2.3572],
         [ 0.8521, -0.0000,  0.7996,  ..., -1.5744,  3.0043, -0.0262]]],
       grad_fn=<MulBackward0>)

In [115]:
model.word_embeddings.weight[31]

tensor([-1.4806e+00,  1.1494e+00, -5.8721e-01,  3.4110e-01, -9.6616e-01,
        -6.8044e-01,  1.4798e+00,  1.0209e+00, -6.4485e-01,  4.5467e-01,
         9.7513e-01, -2.6011e-01, -5.5265e-01,  8.2508e-01, -1.4231e+00,
         4.4060e-01,  6.4321e-02, -8.1924e-01,  5.5717e-01,  1.4058e+00,
        -1.2668e-01,  1.2402e+00, -1.9757e+00, -2.4879e-01, -1.8383e+00,
         4.5399e-01,  7.0577e-01,  4.1375e-01, -3.5001e-01,  9.1271e-01,
        -2.8519e+00, -1.9229e-01,  4.1333e-01,  3.0596e+00,  5.8749e-01,
         1.4367e+00,  2.8893e-01,  1.4455e+00, -6.1530e-01,  8.0917e-01,
        -1.2648e+00, -8.2938e-01, -2.7060e-01, -2.1032e-01, -8.6232e-01,
         2.0441e+00, -1.2790e+00, -4.3685e-01,  1.1931e+00, -2.1604e+00,
        -3.9213e-01,  1.1190e+00,  4.5902e-01,  4.0149e-01, -1.6754e+00,
         1.4853e+00, -8.6224e-01,  7.9496e-01, -2.5201e-01,  9.5908e-01,
        -1.1209e+00, -7.3761e-01, -1.3217e+00, -5.9904e-01,  6.4476e-01,
         2.8471e-01,  1.3390e-01,  3.5054e-01,  5.6

In [120]:
emb = model.word_embeddings(input_ids)
input_ids.shape, emb.shape
emb[0][0]

tensor([-1.4806e+00,  1.1494e+00, -5.8721e-01,  3.4110e-01, -9.6616e-01,
        -6.8044e-01,  1.4798e+00,  1.0209e+00, -6.4485e-01,  4.5467e-01,
         9.7513e-01, -2.6011e-01, -5.5265e-01,  8.2508e-01, -1.4231e+00,
         4.4060e-01,  6.4321e-02, -8.1924e-01,  5.5717e-01,  1.4058e+00,
        -1.2668e-01,  1.2402e+00, -1.9757e+00, -2.4879e-01, -1.8383e+00,
         4.5399e-01,  7.0577e-01,  4.1375e-01, -3.5001e-01,  9.1271e-01,
        -2.8519e+00, -1.9229e-01,  4.1333e-01,  3.0596e+00,  5.8749e-01,
         1.4367e+00,  2.8893e-01,  1.4455e+00, -6.1530e-01,  8.0917e-01,
        -1.2648e+00, -8.2938e-01, -2.7060e-01, -2.1032e-01, -8.6232e-01,
         2.0441e+00, -1.2790e+00, -4.3685e-01,  1.1931e+00, -2.1604e+00,
        -3.9213e-01,  1.1190e+00,  4.5902e-01,  4.0149e-01, -1.6754e+00,
         1.4853e+00, -8.6224e-01,  7.9496e-01, -2.5201e-01,  9.5908e-01,
        -1.1209e+00, -7.3761e-01, -1.3217e+00, -5.9904e-01,  6.4476e-01,
         2.8471e-01,  1.3390e-01,  3.5054e-01,  5.6

In [67]:
pos_emb = model.position_embeddings(input_ids)

In [68]:
tok_emb = model.token_type_embeddings(torch.zeros_like(input_ids))

In [126]:
input_ids

tensor([[31, 51, 12, 23, 99],
        [15,  5,  1,  0,  0]])

In [125]:
if token_type_ids is None:
    token_type_ids = torch.zeros_like(input_ids)
token_type_embeddings = model.token_type_embeddings(token_type_ids)
token_type_embeddings

tensor([[[ 0.7394, -0.1369,  1.4752,  ..., -1.6244,  1.8485,  0.4632],
         [ 0.7394, -0.1369,  1.4752,  ..., -1.6244,  1.8485,  0.4632],
         [ 0.2977,  0.8332, -0.5640,  ..., -0.7253,  0.8034, -0.3100],
         [ 0.2977,  0.8332, -0.5640,  ..., -0.7253,  0.8034, -0.3100],
         [ 0.2977,  0.8332, -0.5640,  ..., -0.7253,  0.8034, -0.3100]],

        [[ 0.7394, -0.1369,  1.4752,  ..., -1.6244,  1.8485,  0.4632],
         [ 0.2977,  0.8332, -0.5640,  ..., -0.7253,  0.8034, -0.3100],
         [ 0.2977,  0.8332, -0.5640,  ..., -0.7253,  0.8034, -0.3100],
         [ 0.2977,  0.8332, -0.5640,  ..., -0.7253,  0.8034, -0.3100],
         [ 0.2977,  0.8332, -0.5640,  ..., -0.7253,  0.8034, -0.3100]]],
       grad_fn=<EmbeddingBackward0>)

In [71]:
input_ids.size()

torch.Size([2, 5])

In [70]:
seq_length = input_ids.size(1)
seq_length

5

In [121]:
seq_length = input_ids.size(1) # 문장 길이
position_ids = torch.arange(
    seq_length, dtype=torch.long, device=input_ids.device)
seq_length, position_ids

(5, tensor([0, 1, 2, 3, 4]))

In [79]:
position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
position_ids

tensor([0, 1, 2, 3, 4])

In [122]:
position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
position_ids.shape, position_ids

(torch.Size([2, 5]),
 tensor([[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]]))

In [124]:
position_embeddings = model.position_embeddings(position_ids)
position_embeddings

tensor([[[-0.1813,  0.0983, -0.8991,  ..., -0.1996,  0.3542,  1.7562],
         [-0.2457, -0.2471, -0.0510,  ..., -2.1774,  0.8956, -0.6549],
         [-0.5985,  1.6512, -0.8037,  ...,  1.5534,  0.1627,  2.2195],
         [-1.6036,  0.4944,  0.6576,  ...,  1.2255,  0.8074,  2.3766],
         [ 0.2873, -0.4889, -0.5152,  ..., -0.4286,  1.9095, -0.5513]],

        [[-0.1813,  0.0983, -0.8991,  ..., -0.1996,  0.3542,  1.7562],
         [-0.2457, -0.2471, -0.0510,  ..., -2.1774,  0.8956, -0.6549],
         [-0.5985,  1.6512, -0.8037,  ...,  1.5534,  0.1627,  2.2195],
         [-1.6036,  0.4944,  0.6576,  ...,  1.2255,  0.8074,  2.3766],
         [ 0.2873, -0.4889, -0.5152,  ..., -0.4286,  1.9095, -0.5513]]],
       grad_fn=<EmbeddingBackward0>)

In [81]:
pos_emb = model.position_embeddings(position_ids)
pos_emb

tensor([[[-0.1813,  0.0983, -0.8991,  ..., -0.1996,  0.3542,  1.7562],
         [-0.2457, -0.2471, -0.0510,  ..., -2.1774,  0.8956, -0.6549],
         [-0.5985,  1.6512, -0.8037,  ...,  1.5534,  0.1627,  2.2195],
         [-1.6036,  0.4944,  0.6576,  ...,  1.2255,  0.8074,  2.3766],
         [ 0.2873, -0.4889, -0.5152,  ..., -0.4286,  1.9095, -0.5513]],

        [[-0.1813,  0.0983, -0.8991,  ..., -0.1996,  0.3542,  1.7562],
         [-0.2457, -0.2471, -0.0510,  ..., -2.1774,  0.8956, -0.6549],
         [-0.5985,  1.6512, -0.8037,  ...,  1.5534,  0.1627,  2.2195],
         [-1.6036,  0.4944,  0.6576,  ...,  1.2255,  0.8074,  2.3766],
         [ 0.2873, -0.4889, -0.5152,  ..., -0.4286,  1.9095, -0.5513]]],
       grad_fn=<EmbeddingBackward0>)

In [83]:
emb + pos_emb + tok_emb

tensor([[[-0.9225,  1.1107, -0.0111,  ..., -2.3113,  2.5776,  1.0327],
         [ 0.4528,  1.4399,  0.8410,  ..., -4.3547,  3.6022, -1.0994],
         [ 0.6467,  0.2667, -0.0129,  ...,  0.8187,  2.8086,  3.1922],
         [ 1.5365, -0.7661,  1.9692,  ..., -0.5671,  2.0402,  2.0562],
         [ 0.8985, -0.8304,  1.7427,  ..., -0.9196,  3.9093,  0.1902]],

        [[ 0.6833,  0.4515,  0.7226,  ..., -2.7106,  2.2928,  3.3426],
         [ 0.4065, -0.2599, -0.2544,  ..., -4.7195,  4.0781, -0.2462],
         [ 0.0629,  1.6372, -0.0887,  ...,  1.0221,  1.7888,  4.6532],
         [-0.8642,  0.3574,  2.1328,  ..., -0.3988,  2.6559,  2.8398],
         [ 1.0267, -0.6259,  0.9600,  ..., -2.0530,  3.7580, -0.0881]]],
       grad_fn=<AddBackward0>)

In [110]:
nn.Embedding?

## 8.2.5 BertLayer 모듈
- BertLayer는 Transformer부분에 해당.
- 서브 네트워크로서 Self-Attention을 계산하는 BertAttention과 Self-Attention의 출력을 처리하는 전결합 층인 BertIntermediate, 그리고 Self-Attention 출력과 BertIntermediate에서 처리한 특징량을 더하는 BertOutput 세 가지로 구성됩니다.
- BertLayer에 대한 입력은 Embedding 모듈의 출력 또는 앞단의 BertLayer에서의 출력이며 크기는 (batch_size, seq_len, hidden_size)입니다. - BertLayer구현에서 7장 Transformer와 두 가지 다른 점이 있습니다.
- 첫째, BertIntermediate 전결합 층 뒤의 활성화 함수에 GELU함수를 사용하는 점입니다. GELU는 기본적으로 RELU와 같은 형태의 함수입니다. 입력이 0이지만 ReLU출력이 거친(매끄러운 변화가 아니라 급격환 변화) 반면 GELU는 입력 0 근처의 출력이 매끄러운 형태입니다.
- 둘쩨, Attention이 Multi-Headed Self-Attention입니다. Transformer도 Multi-Headed Self-Attention이지만 7장에서는 이해를 돕기 위하여 단일 Self-Attention으로 구현하였습니다. Multi-Headed Self-Attention은 단순히 Self-Attention이 여러 개 있는것 뿐입니다.

In [88]:
class BertLayer(nn.Module):
    '''BERT의 BertLayer모듈이다. Transformer가 된다.'''
    
    def __init__(self, config):
        super(BertLayer, self).__init__()
        
        # Self-Attention 부분
        self.attention = BertAttention(config)
        
        # Self-Attention의 출력을 처리하는 전결합 층
        self.intermediate = BertIntermediate(config)
        
        # Self-Attention에 의한 특징량과 BertLayer에 원래의 입력을 더하는 층
        self.output = BertOutput(config)
        
    def forward(self, hidden_states, attention_mask, attention_show_fig=False):
        '''
        hidden_states : Embedder 모듈의 출력 텐서 [batch_size, seq_len, hidden_size]
        attention_mask : Transformer의 마스크와 같은 기능의 마스킹
        attention_show_fig : Self-Attention의 가중치를 반환할지의 플래그
        '''
        if attention_show_fig == True:
            '''attention_show일 경우 attention_probs도 반환한다.'''
            attention_output, attention_probs = self.attention(hidden_states, 
                                                               attention_mask, 
                                                               attention_show_fig)
            
            intermediate_output = self.intermediate(attention_output)
            
            layer_output = self.output(intermediate_output, attention_output)
            
            return layer_output, attention_probs
        
        elif attention_show_fig == False:
            attention_output = self.attention(hidden_states, 
                                              attention_mask, 
                                              attention_show_fig)
            
            intermediate_output = self.intermediate(attention_output)
            
            layer_output = self.output(intermediate_output, attention_output)
            
            return layer_output # [batch_size, seq_length, hidden_size]
        
class BertAttention(nn.Module):
    '''BertLayer 모듈의 Self-Attention 부분'''
    def __init__(self, config):
        super(BertAttention, self).__init__()
        self.selfattn = BertSelfAttention(config)
        self.output = BertSelfOutput(config)
        
    def forward(self, input_tensor, attention_mask, attention_show_fig=False):
        '''
        input_tensor : Embeddings 모듈 또는 앞단의 BertLayer에서의 출력
        attention_mask : Transformer의 마스크와 같은 기능의 마스킹
        attention_show_fig : Self-Attention의 가중치를 반환할지의 플래그
        '''
        if attention_show_fig == True:
            '''attention_show일 경우 attention_probs도 반환한다.'''
            self_output, attention_probs = self.selfattn(input_tensor, attention_mask, attention_show_fig)
            attention_output = self.output(self_output, input_tensor)
            return attention_output, attention_probs
        
        elif attention_show_fig == False:
            self_output = self.selfattn(input_tensor, attention_mask, attention_show_fig)
            attention_output = self.output(self_output, input_tensor)
            return attention_output
        
class BertSelfAttention(nn.Module):
    '''BertAttention의 Self-Attention이다'''
    
    def __init__(self, config):
        super(BertSelfAttention, self).__init__()
        
        self.num_attention_heads = config.num_attention_heads
        # num_attention_heads = 12
        
        self.attention_head_size = int(
            config.hidden_size / config.num_attention_heads) # 768 / 12 = 64
        self.all_head_size = self.num_attention_heads * \
            self.attention_head_size # = 'hidden_size' : 768
        
        # Self-Attention의 특징량을 작성하는 전결합 층
        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)
        
        # drop out
        self.drop_out = nn.Dropout(config.attention_probs_dropout_prob)
        
    def transpose_for_scores(self, x):
        '''Multi-Headed Attention용으로 텐서의 형태 변환
        [batch_size, seq_len, hidden] -> [batch_size, 12, seq_len, hidden/12]
        '''
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)
    
    def forward(self, hidden_states, attention_mask, attention_show_fig=False):
        '''
        hidden_states : Embeddings 모듈 또는 앞단의 BertLayer에서의 출력
        attention_mask : Transformer의 마스크와 같은 기능의 마스킹
        attention_show_fig : Self-Attention의 가중치를 반환할지 플래그
        '''
        
        # 입력의 전결합 층에서 특징량 변환(Multi-headed Attention 전부 한꺼번에 변환)
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)
        
        # Multi-Headed Attention용으로 텐서 형태 변환
        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)
        
        # 특징량끼리 곱하여 비슷한 정도를 Attention_scores로 구한다.
        attention_scores = torch.matmul(
            query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / \
            math.sqrt(self.attention_head_size)
        
        # 마스크가 있는 부분에 마스크 적용
        attention_scores = attention_scores + attention_mask
        # 마스크는 곰셈이 아니라 덧셈이 직관적이지만 그 후에 소프트맥스로 정규화하므로
        # 마스크된 부분은 -inf로 한다. attention_mask에는 원래 0이나 -inf가 있으므로 덧셈으로 한다.
        
        # Attention 정규화
        attention_probs = nn.Softmax(dim=-1)(attention_scores)
        
        # 드롭아웃
        attention_probs = self.drop_out(attention_probs)
        
        # Attention Map을 곱한다.
        context_layer = torch.matmul(attention_probs, value_layer)
        
        # Multi-Headed Attention의 텐서 형태를 원래대로 되돌린다.
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        
        # attention_show일 경우 attention_probs도 반환
        if attention_show_fig == True:
            return context_layer, attention_probs
        elif attention_show_fig == False:
            return context_layer
        
class BertSelfOutput(nn.Module):
    '''BertSelfAttention의 출력을 처리하는 전결합 층이다'''
    
    def __init__(self, config):
        super(BertSelfOutput, self).__init__()
        
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # 'hidden_dropout_prob': 0.1
        
    def forward(self, hidden_states, input_tensor):
        '''
        hidden_stats : BertSelfAttention의 출력 텐서
        input_tensor : Embeddings 모듈 또는 앞단의 BertLayer에서의 출력
        '''
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states
    
def gelu(x):
    '''Gaussian Error Linear Unit라는 활성화 함수이다
    ReLU가 0으로 거칠고 불연속적이므로 연속적으로 매끄럽게 한 셩태의 ReLU이다
    '''
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))
        
class BertIntermediate(nn.Module):
    '''BERT의 TransformerBlock 모듈 FeedForward'''
    
    def __init__(self, config):
        super(BertIntermediate, self).__init__()
        
        # 전결합 층 : 'hidden_size': 768, 'intermediate_size': 3072
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        # 활성화 함수
        self.intermediate_act_fn = gelu
        
    def forward(self, hidden_states):
        '''
        hidden_states : BertAttention의 출력
        '''
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states) # GELU에 의한 활성화
        return hidden_states
    
class BertOutput(nn.Module):
    '''BERT의 TransformerBlock 모듈 FeedForward'''
    
    def __init__(self, config):
        super(BertOutput, self).__init__()
        
        # 전결합 층 : 'intermediate_size': 3072, 'hidden_size': 768
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        
        # 'hidden_dropout_prob': 0.1
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, hidden_states, input_tensor):
        '''
        hidden_states : BertIntermediate 출력 텐서
        input_tensor : BertAttention 출력 텐서
        '''
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

## 8.2.6 BertLayer 모듈의 반복 부분
- BERT_Base에서는 BertLayer 모듈(Transformer)을 12회 반복.
- 이들을 묶어서 BertEncoder클래스로 만듭니다.
- 단순히 BertLayer 12개를 nn.ModuleList에 기재하여 순전파.
- 순전파 함수 forward의 인수
    - output_all_encoded_layer인수는 반환 값으로 BertLayer에서 출력된 특징량을 12단만큼 모두 반환할지 아니면 12단 최종 층의 특징량만 반환할지 여부를 지정하는 변수.
    - 12단의 Transformer 중간에 단어 벡터가 어떻게 변해가는지 확인하고 싶을 때 output_all_encoded_layers인수를 True로 하여 12단 만큼의 단어 벡터를 꺼낼 수 있습니다.
    - 단순히 12단 출력만을 사용하여 자연어 처리를 작업하는 경우 False로 하여 최종 BertLayer 모듈 출력만 BertEncoder에서 출력시킨 후 사용
    - attention_show_fig인수는 BertLayer 모듈에서 사용했던 변수와 동일
    - Self-Attention의 가중치를 출력할지 여부를 지정합니다. BERT_Base의 Attention은 각 층이 12개인 Multi-headed Self-Attention입니다.
    - BertEncoder에서 attention_show_fig인수를 True로 한 경우에는 BertLayer 모듈 중 12단 끝에 있는 BertLayer 모듈에서 12개의 Multi-Headed Self-Attention가중치를 출력합니다.

In [89]:
# BertLayer 모듈의 반복 부분이다.
class BertEncoder(nn.Module):
    def __init__(self, config):
        '''BertLayer 모듈의 반복 부분'''
        super(BertEncoder, self).__init__()
        
        # config.num_hidden_layers의 값, 즉 12개의 BertLayer 모듈을 만든다.
        self.layer = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])
        
    def forward(self, hidden_states, attention_mask, output_all_encoded_layers=True, attention_show_fig=False):
        '''
        hidden_states : Embeddings 모듈 출력
        attention_mask : Transformer의 마스크와 동일한 기능의 마스킹
        output_all_encoded_layers : 반환 값을 전체 TransformerBlock 모듈의 출력으로 할지 마지막 층만으로 한정할지의 플래그
        attention_show_fig : Self-Attention의 가중치를 반환할지의 플래그
        '''
        
        # 반환 값으로 사용할 리스트
        all_encoder_layers = []
        
        # BertLayer 모듈의 처리 반복
        for layer_module in self.layer:
            
            if attention_show_fig == True:
                '''attention_show의 경우 attention_probs도 반환'''
                hidden_states, attention_probs = layer_module(
                    hidden_states, attention_mask, attention_show_fig)
            elif attention_show_fig == False:
                hidden_states = layer_module(
                    hidden_states, attention_mask, attention_show_fig)
                
            # 반환 값으로 BertLayer에서 출력된 특징량만을 사용할 경우의 처리
            if output_all_encoded_layers:
                all_encoder_layers.append(hidden_states)
                    
        # 반환 값으로 마지막 BertLayer에서 출력된 특징량만을 사용할 경우의 처리
        if not output_all_encoded_layers:
            all_encoder_layers.append(hidden_states)
            
        # attention_show의 경우 attention_probs(마지막 12단)도 반환한다.
        if attention_show_fig == True:
            return all_encoder_layers, attention_probs
        elif attention_show_fig == False:
            return all_encoder_layers

## 8.2.7 BertPooler 모듈
- BertPooler 모듈은 BertEncoder출력에서 입력 문장의 첫 번째 단어인 [CLS]부분의 특징량 텐서(1 x 768차원)을 꺼내 전결합 층을 사용한 후 특징량을 변환하는 모듈입니다.
- 전결합 층 뒤에 활성화 함수 Tanh을 사용하고 출력을 1에서 -1까지 범위로 합니다. 출력 텐서의 크기는 (batch_size, hidden_size)입니다. 

In [90]:
class BertPooler(nn.Module):
    '''입력 문장의 첫 번째 단어 [cls]의 특징량을 반환하고 유지하기 위한 모듈'''
    def __init__(self, config):
        super(BertPooler, self).__init__()
        
        # 전결합 층, 'hidden_size':768
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()
        
    def forward(self, hidden_states):
        # 첫 번째 단어의 특징량 취득
        first_token_tensor = hidden_states[:, 0]
        
        # 전결합 층에서 특징량 변환
        pooled_output = self.dense(first_token_tensor)
        
        # 활성화 함수 Tanh을 계산
        pooled_output = self.activation(pooled_output)
        
        return pooled_output

## 8.2.8 동작 확인
- 미니 배치의 크기를 2, 각 미니 배치의 문장 길이르 5로 하여 입력을 적당히 생성
- 길이 5에 두 문장이 포함되어 있음. 어떠한 단어까지 첫 번째 문장이고 어떠한 단어부터 두 번째 문장인지 나타내는 문장 ID와 Attention용 마스크도 생성. 이러한 입력으로 동작을 확인
- Attention용 마스크를 확장한 extended_attention_mask 변수를 작성한다는 점을 주의해야 한다
- Multi-Headed Self-Attention에서 Attention마스크를 사용할 수 있도록 하는 변환.
- Attention을 적용하지 않는 부분은 시그모이드를 계산했을 때 0이 되도록 마이너스 무한의 대안으로써 -10000을 대입.

In [91]:
# 동작 확인

# 입력 단어 ID열 batch_size는 두 가지
input_ids = torch.LongTensor([[31, 51, 12, 23, 99], [15, 5, 1, 0, 0]])
print('입력 단어 ID열의 텐서 크기: ', input_ids.shape)
# 마스크
attention_mask = torch.LongTensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
print('입력 마스크의 텐서 크기: ', attention_mask.shape)

# 문장의 ID, 두 미니 배치 각각에 대한 0은 첫 번째 문장을, 1은 두 번째 문장을 나타낸다.
token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])
print('입력 문장 ID의 텐서 크기: ', token_type_ids.shape)

# BERT의 각 모듈 준비
embeddings = BertEmbeddings(config)
encoder = BertEncoder(config)
pooler = BertPooler(config)

# 마스크 변형 [batch_size, 1, 1, seq_length]로 한다.
# Attention을 적용하지 않는 부분은 마이너스 무한으로 하고 위하여 -10000을 곱한다.
extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
extended_attention_mask = extended_attention_mask.to(dtype=torch.float32)
extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
print('확장된 마스크의 텐서 크기: ', extended_attention_mask.shape)

# 순전파
out1 = embeddings(input_ids, token_type_ids)
print('BertEmbeddings의 출력 텐서 크기:', out1.shape)

out2 = encoder(out1, extended_attention_mask)
# out2는 [minibatch, seq_length, embedding_dim]이 12개 리스트
print('BertEncoder 최후 층의 출력 텐서 크기:', out2[0].shape)

out3 = pooler(out2[-1]) # out2는 12층의 특징량 리스트가 되어 가장 마지막을 사용
print('BertPooler의 출력 텐서 크기:', out3.shape)

입력 단어 ID열의 텐서 크기:  torch.Size([2, 5])
입력 마스크의 텐서 크기:  torch.Size([2, 5])
입력 문장 ID의 텐서 크기:  torch.Size([2, 5])
확장된 마스크의 텐서 크기:  torch.Size([2, 1, 1, 5])
BertEmbeddings의 출력 텐서 크기: torch.Size([2, 5, 768])
BertEncoder 최후 층의 출력 텐서 크기: torch.Size([2, 5, 768])
BertPooler의 출력 텐서 크기: torch.Size([2, 768])


In [133]:
extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
attention_mask.shape, extended_attention_mask.shape, extended_attention_mask

(torch.Size([2, 5]),
 torch.Size([2, 1, 1, 5]),
 tensor([[[[1, 1, 1, 1, 1]]],
 
 
         [[[1, 1, 1, 0, 0]]]]))

In [134]:
extended_attention_mask = extended_attention_mask.to(dtype=torch.float32)
extended_attention_mask.shape, extended_attention_mask

(torch.Size([2, 1, 1, 5]),
 tensor([[[[1., 1., 1., 1., 1.]]],
 
 
         [[[1., 1., 1., 0., 0.]]]]))

In [135]:
extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
extended_attention_mask.shape, extended_attention_mask

(torch.Size([2, 1, 1, 5]),
 tensor([[[[    -0.,     -0.,     -0.,     -0.,     -0.]]],
 
 
         [[[    -0.,     -0.,     -0., -10000., -10000.]]]]))

In [141]:
out1

tensor([[[ 0.5183,  0.8744, -0.6474,  ..., -0.1523,  0.0000, -1.8388],
         [-0.0000, -2.2031,  0.7122,  ...,  0.2697, -0.0000, -1.0999],
         [-0.5974,  0.7153, -1.2055,  ...,  1.6777,  0.6225,  0.5458],
         [ 1.4920,  0.1230, -0.0000,  ...,  0.1844, -1.1844,  0.2829],
         [-0.4239,  0.1712, -1.7108,  ..., -0.6405, -1.0815,  0.5137]],

        [[ 1.0652, -0.2365, -0.2122,  ...,  1.3648,  1.8276, -0.9468],
         [ 0.7128, -2.1402,  0.8725,  ..., -1.3434, -1.8721, -0.1389],
         [-0.4667,  0.1940, -0.7100,  ...,  0.3417,  0.9378,  0.1089],
         [ 0.0000,  0.5578, -0.6825,  ...,  0.3566, -2.2600,  0.7890],
         [-1.0589, -0.3668, -1.6822,  ..., -1.2145, -1.0332,  1.1162]]],
       grad_fn=<MulBackward0>)

In [140]:
out2 = encoder(out1, extended_attention_mask)
out1.shape, len(out2), out2[0].shape, out2[0]

(torch.Size([2, 5, 768]),
 12,
 torch.Size([2, 5, 768]),
 tensor([[[ 5.0371e-01,  8.2984e-01, -5.0412e-01,  ...,  7.2131e-02,
            1.7786e-01, -2.2644e+00],
          [ 5.6436e-01, -1.9141e+00,  6.7024e-01,  ...,  4.3499e-01,
            4.4968e-01, -9.2028e-01],
          [-1.4776e-01,  9.9116e-01, -1.0737e+00,  ...,  1.8967e+00,
            4.7691e-01,  3.4587e-01],
          [ 1.6951e+00,  4.8224e-01, -1.2518e-01,  ...,  4.8702e-01,
           -1.2383e+00,  2.1447e-01],
          [-2.1283e-01,  7.6516e-01, -1.3443e+00,  ..., -5.4793e-01,
           -9.2073e-01,  3.2209e-01]],
 
         [[ 1.1051e+00, -2.8525e-01,  3.5683e-01,  ...,  1.5262e+00,
            1.9065e+00, -1.0319e+00],
          [ 6.1323e-01, -1.6044e+00,  1.1423e+00,  ..., -8.1751e-01,
           -1.8087e+00, -3.5258e-04],
          [ 5.6175e-02, -1.1095e-01, -2.0454e-01,  ...,  1.0018e+00,
            6.1780e-01, -8.1472e-02],
          [-1.1993e-02,  4.7299e-01, -4.4785e-01,  ...,  8.5274e-01,
           -1.8

In [107]:
softmax = nn.Softmax()
torch.argmax(softmax(out3[0]))

/Users/macos/anaconda3/envs/dl/lib/python3.8/site-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor(510)

## 8.2.9 모두 연결하여 BERT모델로
- 동작을 확인한 후 문제가 없다면 모두 연결한 BERT 모델로 지정.

In [92]:
class BertModel(nn.Module):
    '''모듈을 전부 연결한 BERT 모델'''
    
    def __init__(self, config):
        super(BertModel, self).__init__()
        
        # 세 가지 모듈 작성
        self.embeddings = BertEmbeddings(config)
        self.encoder = BertEncoder(config)
        self.pooler = BertPooler(config)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=True,
                attention_show_fig=False):
        '''
        input_ids : [batch_size, sequence_length] 문장의 단어 ID 나열
        token_type_ids : [batch_size, sequence_length] 각 단어가 첫 번째 문장인지 두 번째 문장인지 나타내는 id
        attention_mask: Transformer의 마스크와 같은 기능의 마스킹
        output_all_encoded_layers : 마지막 출력에 12단의 Transformer 모두 리스트로 반환할지 마지막만인지 지정
        attention_show_fig : Self-Attention의 가중치를 반환할지 플래그
        '''
        
        # Attention 마스크와 첫 번째, 두 번째 문장의 id가 없으면 작성
        if attention_mask is None:
            attention_mask = torch.one_like(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
            
        # 마스크 변형 [minibatch, 1, 1, seq_length]로 한다.
        # 나중에 Multi-Headed Self-Attention에서 사용할 수 있는 형태로 하기 위하여
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        
        # 마스크는 0, 1 이지만 소프트맥스를 계산할 때 마스크가 되도록 0과 -inf로 한다.
        # -inf 대신 -10000으로 한다
        extended_attention_mask = extended_attention_mask.to(dtype=torch.float32)
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        
        # 순전파 시킨다
        # BertEmbeddings 모듈
        embedding_output = self.embeddings(input_ids, token_type_ids)
        
        # BertLayer 모듈(Transformer)을 반복하는 BertEncoder 모듈
        if attention_show_fig == True:
            '''attention_show의 경우 attention_probs도 반환'''
            
            encoded_layers, attention_probs = self.encoder(embedding_output,
                                                           extended_attention_mask,
                                                           output_all_encoded_layers,
                                                           attention_show_fig)
            
        elif attention_show_fig == False:
            encoded_layers = self.encoder(embedding_output,
                                          extended_attention_mask,
                                          output_all_encoded_layers,
                                          attention_show_fig)
            
        # BertPooler 모듈
        # 인코더의 맨 마지막 BertLayer에서 출력된 특징량 사용
        pooled_output = self.pooler(encoded_layers[-1])
        
        # output_all_encoded_layer가 False인 경우는 리스트가 아닌 텐서를 반환
        if not output_all_encoded_layers:
            encoded_layers = encoded_layers[-1]
            
        # attention_show의 경우 attention_probs(가장 마지막)도 반환한다.
        if attention_show_fig == True:
            return encoded_layers, pooled_output, attention_probs
        elif attention_show_fig == False:
            return encoded_layers, pooled_output

In [93]:
final_model = BertModel(config)
final_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (selfattn): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (drop_out): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
         

In [94]:
ti.summary(final_model)

Layer (type:depth-idx)                             Param #
BertModel                                          --
├─BertEmbeddings: 1-1                              --
│    └─Embedding: 2-1                              23,440,896
│    └─Embedding: 2-2                              393,216
│    └─Embedding: 2-3                              1,536
│    └─BertLayerNorm: 2-4                          1,536
│    └─Dropout: 2-5                                --
├─BertEncoder: 1-2                                 --
│    └─ModuleList: 2-6                             --
│    │    └─BertLayer: 3-1                         7,087,872
│    │    └─BertLayer: 3-2                         7,087,872
│    │    └─BertLayer: 3-3                         7,087,872
│    │    └─BertLayer: 3-4                         7,087,872
│    │    └─BertLayer: 3-5                         7,087,872
│    │    └─BertLayer: 3-6                         7,087,872
│    │    └─BertLayer: 3-7                         7,087,872
│    │   

In [27]:
# 동작 확인
# 입력 준비
input_ids = torch.LongTensor([[31, 52, 12, 23, 99], [15, 5, 1, 0, 0]])
attention_mask = torch.LongTensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])

# BERT 모델을 만든다.
net = BertModel(config)

# 순전파
encoded_layers, pooled_output, attention_probs = net(
    input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False, attention_show_fig=True)

print('encoded_layers의 텐서 크기:', encoded_layers.shape)
print('pooled_output의 텐서 크기:', pooled_output.shape)
print('attention_probs의 텐서 크기:', attention_probs.shape)

encoded_layers의 텐서 크기: torch.Size([2, 5, 768])
pooled_output의 텐서 크기: torch.Size([2, 768])
attention_probs의 텐서 크기: torch.Size([2, 12, 5, 5])
